In [46]:
import pandas as pd
from pandas import option_context

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

import re
import string

__IMPORTANT:__ Once you have sorted out the pipeline, remember to do train-test-split to remove your test set

## Main notebook for pre-processing tweets

#### Prepare dataframe

In [47]:
df = pd.read_csv('custom_out2.csv')
df.dropna(axis=1,inplace=True)

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [48]:
## Filter down to non-Revolut tweets (intention is to understand tweets from customers)
df_eng = df[(df['username'] != 'revolutapp') & (df['language'] == 'en')]

#### Clean tweet column

In [49]:
def clean_text(text):
    '''Make text lowercase, remove punctuation and remove reference to revolutapp'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('revolutapp', '', text)
    return text

df_eng['tweet'] = df_eng.tweet.map(clean_text)

<ipython-input-49-53ab4f15f761>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng['tweet'] = df_eng.tweet.map(clean_text)


In [50]:
## Remove emojis
df_eng['tweet'] = df_eng['tweet'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

<ipython-input-50-77c184c069e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng['tweet'] = df_eng['tweet'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))


In [51]:
with option_context('display.max_colwidth', 600):
    display(df_eng.sample(2))

,id,conversation_id,created_at,date,time,timezone,user_id,username,tweet,language,...,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,video,reply_to
173060,1251824240506085378,1251824240506085378,2020-04-19 06:45:31 EDT,2020-04-19,06:45:31,-500,388972545,vitobuttafarro,any idea of when u are gonna fix your systemive been trying to top up my card since the 9th of apriltalked to 3 agentsit still doesnt work kind regards,en,...,[],1,0,0,[],[],https://twitter.com/vitobuttafarro/status/1251824240506085378,False,0,[]
94057,1298956015132643329,1298942465030922240,2020-08-27 08:10:22 EDT,2020-08-27,08:10:22,-500,564105670,mirinquads,i tried for a full 24 hours sadly no response is there something else to do,en,...,[],0,0,0,[],[],https://twitter.com/mirinquads/status/1298956015132643329,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]"


In [52]:
## Remove known observations that will not add value
df_eng = df_eng[(df_eng['tweet'] != ' followed') & (df_eng['tweet'] != 'followed ')]

#### Word tokenization

In [53]:
df_eng['tokens'] = df_eng.tweet.apply(nltk.word_tokenize)

In [54]:
with option_context('display.max_colwidth', 600):
    display(df_eng.sample(5))

,id,conversation_id,created_at,date,time,timezone,user_id,username,tweet,language,...,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,video,reply_to,tokens
148216,1268166900225359880,1268159922014912512,2020-06-03 09:05:25 EDT,2020-06-03,09:05:25,-500,2768107323,harold_moreels,i dont have any response on the app,en,...,0,0,0,[],[],https://twitter.com/harold_moreels/status/1268166900225359880,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]","[i, dont, have, any, response, on, the, app]"
131791,1276169089740931073,1276168637997551617,2020-06-25 11:03:15 EDT,2020-06-25,11:03:15,-500,978519553293672448,adijovschi,americanexpress what about cyprus,en,...,0,0,0,[],[],https://twitter.com/adijovschi/status/1276169089740931073,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}, {'screen_name': 'AmericanExpress', 'name': 'American Express', 'id': '42712551'}]","[americanexpress, what, about, cyprus]"
211370,1227686842415353857,1227686842415353857,2020-02-12 15:12:07 EST,2020-02-12,15:12:07,-500,1030530344,pisanibugeja,can you reopen my revolut account amd reactivate my app its been over 2 months horrible service,en,...,0,0,0,[],[],https://twitter.com/pisanibugeja/status/1227686842415353857,False,0,[],"[can, you, reopen, my, revolut, account, amd, reactivate, my, app, its, been, over, 2, months, horrible, service]"
76261,1311381272971948032,1311381272971948032,2020-09-30 15:03:54 EDT,2020-09-30,15:03:54,-500,184478648,yankeez78,2 exceptions occurred whats going on i cant log in i was being assisted by an agent on live chat when suddenly the app crashed now i cant log in please help theres no revolut update available right now,en,...,1,0,0,[],[],https://twitter.com/Yankeez78/status/1311381272971948032,False,0,[],"[2, exceptions, occurred, whats, going, on, i, cant, log, in, i, was, being, assisted, by, an, agent, on, live, chat, when, suddenly, the, app, crashed, now, i, cant, log, in, please, help, theres, no, revolut, update, available, right, now]"
141569,1270990059089485824,1270990059089485824,2020-06-11 04:03:38 EDT,2020-06-11,04:03:38,-500,302441542,meme12555,since when exactly is 1250 czk equal to 1k eurgbp 1250 czk 50 eur limit per month since august is a joke or hopefully a mistake 12500 would be still 12 but a little bit better could you clarify this please thanks,en,...,1,0,1,[],[],https://twitter.com/meme12555/status/1270990059089485824,False,0,[],"[since, when, exactly, is, 1250, czk, equal, to, 1k, eurgbp, 1250, czk, 50, eur, limit, per, month, since, august, is, a, joke, or, hopefully, a, mistake, 12500, would, be, still, 12, but, a, little, bit, better, could, you, clarify, this, please, thanks]"


#### Note: How to find and correct mispelled words?
Where in the pipeline is best place to do this?

In [34]:
from spellchecker import SpellChecker

spell = SpellChecker()

misspelled = spell.unknown(df_eng['tokens'])

TypeError: float() argument must be a string or a number, not 'list'

Note you will incorporate stop words when making the count vectoriser